In [26]:
# %run Binance_Keys.ipynb

In [27]:
from binance import Client

In [28]:
import pandas as pd
import ta
import ta.momentum
import ta.trend
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import pytz

In [29]:
# Replace 'your_api_key' and 'your_api_secret' with your actual API key and secret
api_key = 'w7eIzNnotSCB81kSE2pREnOSnoI6ThfAa6rYkIUODPCKLh4hJ16dRn7gKmzbELxE'
api_secret = 'xoKLzwrQmOtskAXLBkS4Kjgjzf283S4LH4A8tp77cSsPcBqQRfwkQHuK0J15lVIE'

# Initialize the client
client = Client(api_key, api_secret)

In [30]:
def get_minute_data(symbol, interval, lookback):
    # Assuming `client` is your Binance API client
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + 'min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame.index = frame.index.tz_localize('UTC')  # Ensure the index is in UTC timezone
    frame.index = frame.index.tz_convert('Asia/Kolkata')  # Convert index to IST timezone
    frame = frame.astype(float)
    return frame


In [31]:
df = get_minute_data('BTCUSDT','1m','100')

In [32]:
def apply_technicals(df):
    df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window=14,smooth_window=3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['rsi'] = ta.momentum.rsi(df.Close,window=14)
    df['macd'] = ta.trend.macd_diff(df.Close)
    df.dropna(inplace=True)

In [33]:
apply_technicals(df)

In [34]:
df.tail()

,Open,High,Low,Close,Volume,%K,%D,rsi,macd
Time,,,,,,,,,
2024-05-11 11:40:00+05:30,60962.00,60962.00,60953.22,60957.99,5.77074,82.354668,85.119397,60.997058,3.560038
2024-05-11 11:41:00+05:30,60958.00,60958.00,60948.00,60948.01,4.56852,67.704052,79.433353,57.508945,2.439425
2024-05-11 11:42:00+05:30,60948.01,60948.01,60948.00,60948.01,3.41216,67.704052,72.587591,57.508945,1.486256
2024-05-11 11:43:00+05:30,60948.00,60948.54,60948.00,60948.53,3.61276,68.467410,67.958505,57.655272,0.717803
2024-05-11 11:44:00+05:30,60948.54,60948.54,60948.53,60948.53,0.33647,68.467410,68.212958,57.655272,0.070587


In [35]:
class Signals:
    def __init__(self, minute_data_df, lags):
        self.minute_data_df = minute_data_df
        self.lags = lags

    def get_trigger(self):
        dfx = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.minute_data_df['%K'].shift(i) < 20) & (self.minute_data_df['%D'].shift(i) < 20)
            dfx[i] = mask.astype(int)
        return dfx.sum(axis=1)

    def decide(self):
        self.minute_data_df['trigger'] = np.where(self.get_trigger(), 1, 0)
        self.minute_data_df['Buy'] = np.where(self.minute_data_df.trigger &
                                              (self.minute_data_df['%K'].between(20, 80)) &
                                              (self.minute_data_df['%D'].between(20, 80)) &
                                              (self.minute_data_df['rsi'] > 50) &
                                              (self.minute_data_df['macd'] > 0), 1, 0
                                              )

In [36]:
inst = Signals(df,25)

In [37]:
inst.decide()

In [38]:
df[df.Buy == 1 ]

,Open,High,Low,Close,Volume,%K,%D,rsi,macd,trigger,Buy
Time,,,,,,,,,,,
2024-05-11 11:24:00+05:30,60910.01,60910.01,60902.63,60902.64,2.48098,65.808226,76.441893,54.289120,2.428857,1,1
2024-05-11 11:25:00+05:30,60902.63,60907.02,60902.63,60907.02,3.53471,71.283910,70.704671,55.645097,1.941808,1,1
2024-05-11 11:27:00+05:30,60919.99,60920.00,60908.29,60908.30,6.80970,72.884111,77.226320,54.874278,1.600417,1,1
2024-05-11 11:28:00+05:30,60908.29,60910.00,60907.26,60910.00,4.79720,75.009376,78.468142,55.414609,1.116197,1,1
2024-05-11 11:32:00+05:30,60939.16,60939.16,60914.27,60914.28,12.41613,46.465077,74.214461,53.035217,1.746835,1,1
2024-05-11 11:33:00+05:30,60914.28,60914.28,60910.63,60910.64,6.18070,41.297558,56.511452,51.845035,0.271340,1,1


In [39]:
def strategy(pair,qty,open_postion=False):
    df = get_minute_data(pair,'1m','100')
    apply_technicals(df)
    inst = Signals(df,25)
    inst.decide()
    print(f"Current price is :"+str(df.Close.iloc[-1]))
    if df.Buy.iloc[-1] == 1:
        order = client.create_order(symbol=pair, side='BUY', type='MARKET', quantity=qty)
        print(order)
        buy_price = float(order['fills'][0]['price'])
        open_postion = True
    while open_postion:
        time.sleep(0.5)
        df=get_minute_data(pair,'1m','2')
        print(f"Current Close price is : " + str(df.Close.iloc[-1]))
        print(f"Current Target price is : "+ str(buy_price * 1.030))
        print(f"Current Stop price is : " + str(buy_price * 0.990))
        if df.Close[-1] <= buy_price * 0.990 or df.Close[-1] >= buy_price * 1.030:
            order = client.create_order(symbol=pair, side='SELL', type='MARKET', quantity=qty)
            print(order)
            break


In [46]:
while True:
    strategy('BTCUSDT',1)
    time.sleep(1)

Current price is :61042.01
Current price is :61042.01
Current price is :61042.0
Current price is :61042.01
Current price is :61042.0
Current price is :61042.01
Current price is :61042.01
Current price is :61042.01
Current price is :61038.62
Current price is :61038.62
Current price is :61038.61
Current price is :61038.62
Current price is :61038.61
Current price is :61034.65
Current price is :61033.93
Current price is :61033.93
Current price is :61032.0
Current price is :61030.88
Current price is :61030.88
Current price is :61030.0
Current price is :61030.0
Current price is :61026.33
Current price is :61026.32
Current price is :61021.83
Current price is :61021.83
Current price is :61021.84
Current price is :61016.38
Current price is :61016.38
Current price is :61016.38
Current price is :61016.38
Current price is :61016.38
Current price is :61016.39
Current price is :61016.39
Current price is :61016.39
Current price is :61016.39
Current price is :61016.39
Current price is :61016.38
Curren

BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.